<a href="https://colab.research.google.com/github/mequanent/Audio-Processing/blob/main/Audio_Processing_Beginner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#pip install librosa -q

In [1]:
import librosa

In [9]:
y # one dimensional array of size 2643264
sr # 22050 Sampling rate
beat_frames # size 212
tempo # 107.666015625
beat_times.size # size 212

(212,)

In [ ]:
# Beat tracking example
import librosa

# 1. Get the file path to an included audio example
filename = librosa.example('nutcracker')


# 2. Load the audio as a waveform `y`
#    Store the sampling rate as `sr`
y, sr = librosa.load(filename)

# 3. Run the default beat tracker
tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)

print('Estimated tempo: {:.2f} beats per minute'.format(tempo))

# 4. Convert the frame indices of beat events into timestamps
beat_times = librosa.frames_to_time(beat_frames, sr=sr)

Estimated tempo: 107.67 beats per minute


In [17]:
# Feature extraction example
import numpy as np
import librosa

# Load the example clip
y, sr = librosa.load(librosa.ex('nutcracker'))

# Set the hop length; at 22050 Hz, 512 samples ~= 23ms
hop_length = 512

# Separate harmonics and percussives into two waveforms
y_harmonic, y_percussive = librosa.effects.hpss(y)

# Beat track on the percussive signal # 107.666015625 , 211 respectively
tempo, beat_frames = librosa.beat.beat_track(y=y_percussive, sr=sr)

# Compute MFCC features from the raw signal # (13, 5163)
mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=13)

# And the first-order differences (delta features) # (13, 5163)
mfcc_delta = librosa.feature.delta(mfcc)

# Stack and synchronize between beat events
# This time, we'll use the mean value (default) instead of median # (26, 212)
beat_mfcc_delta = librosa.util.sync(np.vstack([mfcc, mfcc_delta]), beat_frames)

# Compute chroma features from the harmonic signal # (12, 5163)
chromagram = librosa.feature.chroma_cqt(y=y_harmonic, sr=sr)

# Aggregate chroma features between beat events
# We'll use the median value of each feature between beat frames # (12, 212)
beat_chroma = librosa.util.sync(chromagram, beat_frames, aggregate=np.median)

# Finally, stack all beat-synchronous features together # (38, 212)
beat_features = np.vstack([beat_chroma, beat_mfcc_delta])

array([ 2.2716861e-06,  5.3327208e-06, -7.2473290e-06, ...,
        1.1170751e-05,  1.2871889e-06,  5.4120628e-06], dtype=float32)

In [37]:
# Finally, stack all beat-synchronous features together # (38, 212)
beat_features = np.vstack([beat_chroma, beat_mfcc_delta])
beat_features

array([[ 0.30254456,  0.0334522 ,  0.1058374 , ...,  0.15276003,
         0.07555082,  0.0814149 ],
       [ 0.25782844,  0.04074808,  0.08428392, ...,  0.09633808,
         0.12807782,  0.0932549 ],
       [ 0.3997867 ,  0.0575659 ,  0.09429981, ...,  0.44611084,
         0.186589  ,  0.13245766],
       ...,
       [-0.06548901, -0.14644985,  0.05339273, ..., -0.16999006,
         0.62217766,  0.12908453],
       [-0.02321963, -0.13737966, -0.01143883, ..., -0.24983512,
         0.2491345 ,  0.00830472],
       [ 0.02864804, -0.20102443, -0.05542237, ...,  0.07502007,
        -0.61561984,  0.07628296]], dtype=float32)

In [38]:
beat_features.shape

(38, 212)